In [21]:
import glob, openpyxl, pandas as pd, numpy as np
from datetime import datetime, timedelta
import sqlite3

In [22]:
## Function to read patient's information
def get_Patient_Info(sheet,data):
    
    #print("start get patient info")
    # Create a list to save patient's information
    item = []
    
    # Read cell values in row 7 to the list
    for col in range(1,8):
        value = sheet.cell(row = 7,column = col).value
        if value is not None:
            if is_number(value):
                item.append(value)
            if (value == 'F' or value == 'M'):
                item.append(value)
            if (value == 'Male'):
                item.append('M')
            if (value == 'Female'):
                item.append('F')
          
    # Read cell values in row 8 to the list
    for col in range(1,13):
        value = sheet.cell(row = 8,column = col).value
        if value is not None:
                if is_number(value):
                    item.append(value)
   
    # Create a new list to save patient information 
    data = append_Multi_Item(data,item)
    #print(data)
    
    #print("end get patient info")
    return data

In [23]:
## Function to append multiple item in a item list 
def append_Multi_Item(data, itemList):
    #print("start append_multi_item")
    for i in range(0,(len(itemList))):
        data.append(itemList[i])
    #print("end append_multi_item")
    return data

In [24]:
## Function to check the value is numeric or not
def is_number(s):
    #print("start is_number")
    try:
        float(s)
        #print("end is_number")
        return True
    except ValueError:
        #print("end is_number")
        return False

In [25]:
def get_start_of_data(sheet):
    
    #print("start get_start_of_data")
    # Save the index of the number of start line of raw data
    for row in range(27,32):
        value = sheet.cell(row = row, column = 1).value
        if value == 'min  ':
            data_start = row + 2
            break
    #print("end get_start_of_data")
    return data_start

In [26]:
def get_end_of_data(sheet):
    
    #print("start get_end_of_data")
    # Get max line
    max_row = sheet.max_row
    
    
# Save the index of the numebr of end line of raw data
    try: 
        #print(max_row)
        cell_5 = sheet.cell(row = max_row, column = 5)
        #cell_6 = sheet.cell(row = max_row, column = 6)
        if (cell_5.value is not None):
            #print(cell_6.value)
            #print(cell_5.value)
            if (is_number(cell_5.value)):
                data_end = max_row
            else:
                try:
                    cell_5__rowminusone = sheet.cell(row = max_row - 1, column = 5)
                    if (cell_5__rowminusone.value is not None): 
                        #print(cell_5__rowminusone.value)
                        if (is_number(cell_5__rowminusone.value)):
                            data_end = max_row - 1
                except IndexError:
                    #print("....indexError")
                    exit = False
                    for row in range(max_row - 80 , max_row-1):
                        #print(row)
                        try: 
                            cell = sheet.cell(row = row, column = 2)
                            if (cell.value is None):
                                #print("isnone")
                                #print(row)
                                if (is_data_end(sheet, row-1)):
                                    #print(row - 1)
                                    data_end = row - 1
                                    exit = True
                                if (exit):
                                    break
                        except IndexError:
                            #print("little indexerror")
                            data_end = row - 1
                            exit = True
                        if (exit):
                            break
        else: 
            #print("else")
            exit = False
            try:
                #rint("try/...")
                cell_5__rowminusone = sheet.cell(row = max_row - 1, column = 5)
                #print("cell5")
                if (cell_5__rowminusone.value is not None): 
                    #print(cell_5__rowminusone.value)
                    if (is_number(cell_5__rowminusone.value)):
                        data_end = max_row - 1
                else:
                    for row in range(max_row - 80 , max_row-1):
                        #print(row)
                        try: 
                            cell = sheet.cell(row = row, column = 2)
                            if (cell.value is None):
                                #print("isnone")
                                #print(row)
                                if (is_data_end(sheet, row-1)):
                                    #print(row - 1)
                                    data_end = row - 1
                                    exit = True
                                if (exit):
                                    break
                        except IndexError:
                            #print("little indexerror")
                            data_end = row - 1
                            exit = True
                        if (exit):
                            break
            except IndexError:
                #print("indexError")
                exit = False
                for row in range(max_row - 80 , max_row):
                    #print(row)
                    try: 
                        cell = sheet.cell(row = row, column = 1)
                        if (cell.value is None):
                            #print("isnone")
                            #print(row)
                            if (is_data_end(sheet, row-1)):
                                #print(row - 1)
                                data_end = row - 1
                                exit = True
                            if (exit):
                                break
                    except IndexError:
                        #print("little indexerror")
                        data_end = row - 1
                        exit = True
                    if (exit):
                        break
    except IndexError: 
        #print("big indexerror")
        exit = False
        for row in range(max_row-25, max_row):
            try:
                #print(row)
                cell = sheet.cell(row = row, column = 1)
            except IndexError: 
                data_end = row - 1
                exit = True
            if (exit):
                break
    #print("end get_end_of_data")
    return data_end

In [27]:
def get_data(sheet, data, df, columns):
    
    #print("start get_data")
    # Get the start line of raw data
    data_start = get_start_of_data(sheet)
    
    # Get the end line of raw data
    data_end = get_end_of_data(sheet)
    
    row_count = 0
    # Read data in excel by row and append the data to data list and df
    for row in range(data_start, data_end + 1):
        row_count = row_count + 1
        #print(row_count)
        data_copy = []
        data_copy = append_Multi_Item(data_copy, data)
        #print(data_copy)
        #print(row)
        data_excel = get_data_in_row(sheet, row)
        data_copy = append_Multi_Item(data_copy, data_excel)
        
#         value = data_excel[0]
#         value = datetime.strptime(value,"%M:%S")
#         time = timedelta(minutes = value.minute, seconds = value.second)
#         if row == data_start:
#             test_start = time
#             five_minute = timedelta(minutes = 5, seconds = 0)
#             yellow_start_time = (test_start + five_minute).seconds
#             ten_minute = timedelta(minutes = 10, seconds = 0).seconds
#             yellow_end_time = yellow_start_time + ten_minute
#             #print(yellow_start_time)
#             #print(yellow_end_time)
            
#         # Determine if yellow region starts
#         time_in_sec = time.seconds
#         if (time_in_sec >= yellow_start_time) and (time_in_sec <= yellow_start_time + 15):
#             #print("in if (starts)")
#             yellow_start_row = row_count
            
#         # Determine if yellow region ends
#         if (time_in_sec >= yellow_end_time) and (time_in_sec <= yellow_end_time + 15):
#             #print("in if (ends)")
#             yellow_end_row = row_count

        #print(data)
        #data_copy = append_Multi_Item(data_copy, yellow)
        #print(data_copy)
        #print(data)
        df2 = pd.DataFrame([data_copy], columns = columns)
        df = df.append(df2)
    
#     yellow_start_list = create_list(yellow_start_row, row_count)
#     yellow_end_list = create_list(yellow_end_row, row_count)
    
#     df['Calculate Start Row'] = yellow_start_list
#     df['Calculate End Row'] = yellow_end_list
    
    #print("end get_data")
    return df

In [28]:
def find_yellow(data):
    
    # get length of dataframe
    length = len(data)
    
    # calcualte overall test time
    overall_time = get_test_duration(data)
    
    # time list
    time = []
    for index in range(0, len(df)):
        time_point = datetime.strptime(df['Time (min)'].values[index],"%M:%S")
        time_point = timedelta(minutes = time_point.minute, seconds = time_point.second)
        time_point = time_point.seconds 
        time.append(time_point)
  
    # Find scan_start
    scan_start = 0
    row_count = 0
    for i in range(0, len(time)):
        row_count = row_count + 1
        if time[i] >= 600:
            scan_start = row_count
            break
    # Find scan_end
    scan_end = 0
    row_count = scan_start
    for i in range(scan_start+1, len(time)):
        row_count = row_count+1
        if time[i] >= (time[len(time)-1] - 600):
            scan_end = row_count
            break
    
    yellow_start = scan_start
    yellow_end = scan_end
    
    # if test time is sufficient
    if overall_time >= 1500
        
        time_end = time[scan_start] + 600
        
    else: 

SyntaxError: unexpected EOF while parsing (<ipython-input-28-80c273a8680e>, line 3)

In [40]:
def get_test_duration(df):
    
    start_time = datetime.strptime(df['Time (min)'].values[0],"%M:%S")
    delta_start = timedelta(minutes=start_time.minute, seconds=start_time.second)
    end_time = datetime.strptime(df['Time (min)'].values[-1],"%M:%S")
    delta_end = timedelta(minutes=end_time.minute, seconds=end_time.second)
    deltatime = (delta_end - delta_start).seconds

    return deltatime

In [29]:
def create_list(item, count):
    
    data = []
    
    for index in range(0, count):
        data.append(item)
    
    return data

In [30]:
def get_data_in_row(sheet, row_number):
    
    #print("start get_data_in_row")
    data = []
    #Get and save the data in row
    for col in range(1, 13):
        if col == 1:
            value = sheet.cell(row = row_number, column = col).value
            value = value.strip()
            data.append(value)
        else:
            value = sheet.cell(row = row_number, column = col).value
            if col == 4 or col == 6:
                value = data[col-2] * 1000
            data.append(value)
    
    #print("end get_data_in_row")
    #data.append(yellow_start_row)
    #data.append(yellow_start_row)
    return data

In [31]:
def is_data_end(sheet, row):
    #print("start is_data_end")
    cell_9 = sheet.cell(row = row, column = 9)
    cell_10 = sheet.cell(row = row, column = 10)
    if (is_number(cell_9.value) and is_number(cell_10.value)):
        #print("end is_data_end")
        return True
    else:
        #print("end is_data_end")
        return False

In [32]:
def get_patientID():
    
    sqlite_file = 'RMR_Calculation.sqlite'    # name of the sqlite database file
    table_name = 'Raw_data_to_calculate_RMR'  # name of the table to be created
    column_name = 'Patient_ID' # name of the column
    
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()

    # Creating a new SQLite table with 1 column
    patient_id = c.execute('SELECT MAX({cn}) from {tn}'\
        .format(tn=table_name, cn=column_name)).fetchall()[0][0]

    # Committing changes and closing the connection to the database file
    conn.commit()
    conn.close()
    
    if patient_id == None: 
        patient_id = 1
    else: 
        patient_id = patient_id + 1
    
    return patient_id

In [33]:
def calculate_RMR(data):
    
    start = data['Calculate Start Row'].values[0]
    end = data['Calculate End Row'].values[0]
    data_for_calculate = data.iloc[start-1:end]
    
    VO2 = data_for_calculate['VO2 STPD (ml/min)']
    mean_VO2 = np.mean(VO2)
    VCO2 = data_for_calculate['VCO2 STPD (ml/min)']
    mean_VCO2 = np.mean(VCO2)
    rmr = ((3.9 * mean_VO2) + (1.1 * mean_VCO2)) * 1.44
    
    rmr_list = create_list(rmr, len(data))
    
    data['Calculated RMR'] = rmr_list
    
    return data

In [34]:
path = 'P:\\Metabolic Test\\RMR\\src\\Processed Files\\Checked\\'
all_files = glob.glob(path + "*.xlsx")
fname = all_files[0]

In [35]:
## Create Pandas dataframe
columns = ['Patient ID', 'Age', 'Sex','Height (in)', 'Height (cm)', 'Weight (Ib)',
           'Weight (kg)', 'Time (min)', 'VO2/kg STPD (ml/kg/m)', 'VO2 STPD (L/min)',
          'VO2 STPD (ml/min)', 'VCO2 STPD (L/min)', 'VCO2 STPD (ml/min)', 'VE STPD (L/min)', 'RQ', 
           'RR (BPM)', 'VE/VO2 (BT/ST)', 'METS', 'TM SPD(mph)']
           #'Calculate Start Row','Calcualte End Row','Calcualted RMR']
df = pd.DataFrame(columns = columns)

In [36]:
patient_id = get_patientID()
# Initiation of a data list for row in dataframe
data = [patient_id]
    
# open excel work book and sheet1
wb = openpyxl.load_workbook(fname, use_iterators= True, data_only = True)
sheet_1 = wb.get_sheet_by_name(name = 'Sheet1')
#sheet_2 = wb.get_sheet_by_name(name = 'Subject Report')
    
# Call function to update data list with age, sex, height and weight
data = get_Patient_Info(sheet_1, data)
    
# Call function to get the list of start and end index of yellow region
#yellow = get_yellow_index(start_Time, end_time)
    
# Call function to get the data and save the data to dataframe
df = get_data(sheet_1, data, df, columns)

# find yellow region and add it to dataframe
# df = find

# # Calculate RMR
# df = calculate_RMR(df)

print(df)

    Patient ID   Age Sex  Height (in)  Height (cm)  Weight (Ib)  Weight (kg)  \
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   170.179993   186.380997    84.718636   
0          3.0  46.0   F         67.0   

In [37]:
len(df)

190

In [39]:
df['Time (min)'].values[len(df)-1]

'47:30'

In [43]:
time = df['Time (min)']
time

0     0:18
0     0:30
0     0:49
0     1:03
0     1:20
0     1:35
0     1:46
0     2:08
0     2:16
0     2:32
0     2:49
0     3:03
0     3:16
0     3:40
0     3:47
0     4:06
0     4:15
0     4:30
0     4:48
0     5:01
0     5:18
0     5:32
0     5:49
0     6:04
0     6:17
0     6:34
0     6:52
0     7:00
0     7:16
0     7:32
     ...  
0    40:18
0    40:33
0    40:45
0    41:00
0    41:15
0    41:33
0    41:45
0    42:04
0    42:18
0    42:31
0    42:47
0    43:06
0    43:15
0    43:35
0    43:49
0    44:02
0    44:16
0    44:32
0    44:47
0    45:02
0    45:16
0    45:31
0    45:46
0    46:02
0    46:15
0    46:33
0    46:49
0    47:04
0    47:15
0    47:30
Name: Time (min), dtype: object

In [57]:
time = []
for index in range(0, len(df)):
    time_point = datetime.strptime(df['Time (min)'].values[index],"%M:%S")
    time_point = timedelta(minutes = time_point.minute, seconds = time_point.second)
    time_point = time_point.seconds 
    time.append(time_point)
time

[18,
 30,
 49,
 63,
 80,
 95,
 106,
 128,
 136,
 152,
 169,
 183,
 196,
 220,
 227,
 246,
 255,
 270,
 288,
 301,
 318,
 332,
 349,
 364,
 377,
 394,
 412,
 420,
 436,
 452,
 466,
 483,
 495,
 511,
 526,
 544,
 555,
 571,
 588,
 600,
 618,
 633,
 645,
 669,
 677,
 692,
 709,
 721,
 735,
 753,
 769,
 785,
 795,
 816,
 826,
 844,
 858,
 872,
 889,
 905,
 916,
 930,
 945,
 964,
 979,
 993,
 1005,
 1022,
 1037,
 1052,
 1066,
 1084,
 1097,
 1111,
 1132,
 1141,
 1158,
 1171,
 1189,
 1205,
 1220,
 1232,
 1245,
 1265,
 1280,
 1291,
 1306,
 1326,
 1338,
 1355,
 1365,
 1386,
 1399,
 1415,
 1428,
 1440,
 1455,
 1472,
 1491,
 1502,
 1518,
 1532,
 1548,
 1563,
 1576,
 1595,
 1605,
 1620,
 1638,
 1651,
 1667,
 1681,
 1698,
 1712,
 1726,
 1740,
 1756,
 1771,
 1785,
 1800,
 1817,
 1836,
 1847,
 1864,
 1876,
 1890,
 1909,
 1923,
 1936,
 1951,
 1969,
 1983,
 1997,
 2011,
 2029,
 2042,
 2057,
 2078,
 2090,
 2105,
 2115,
 2132,
 2145,
 2160,
 2179,
 2194,
 2208,
 2221,
 2239,
 2252,
 2265,
 2280,
 2295,
 

In [67]:
 # Find scan_start
scan_start = 0
row_count = 0
for i in range(0, len(time)):
    row_count = row_count + 1
    print(row_count)
    if time[i] >= 600:
        print(row_count)
        scan_start = row_count
        break
# Find scan_end
scan_end = 0
row_count = scan_start
for i in range(scan_start+1, len(time)):
    row_count = row_count+1
    if time[i] >= (time[len(time)-1] - 600):
        scan_end = row_count
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
40


In [68]:
scan_start

40

In [69]:
scan_end

149

In [70]:
time[149]

2252

In [71]:
time[40]

618

In [66]:
time[39]

600